This notebook is for the analysis of experiments in which some flies were interacting with balls that were blocked using magnets. After a certain amount of time, magnets were removed and we want to compare how well they push balls compared to controls.

# Required packages 


In [ ]:
import sys
import os
from icecream import ic

from pathlib import Path

import utils_behavior

from utils_behavior import Ballpushing_utils
from utils_behavior import Utils
from utils_behavior import Processing
from utils_behavior import HoloviewsTemplates

import pandas as pd
import hvplot.pandas
import numpy as np

from scipy import stats
from statsmodels.stats.multitest import multipletests

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

import iqplot

import importlib

import holoviews as hv
import bokeh.io
import colorcet

bokeh.io.output_notebook()

hv.extension("bokeh")

In [ ]:
# Select the experiments to analyze

Let's start with one that is representative of the best settings I had, which was using magnets under the backlighting sheet, keeping control flies in their chambers until the first half-hour of pretraining was finished.
These were done on **240530 and 240531**

In [ ]:
# Get the data path
Datapath = Utils.get_data_path()

# Get all folders with "TNT_Fine" in the name

Folders = [
    f
    for f in os.listdir(Datapath)
    if "240530" in f or "240531" in f and os.path.isdir(Datapath / f)
]

Folders

In [ ]:
importlib.reload(Ballpushing_utils)

In [ ]:
# Generate Experiment objects from each folder

Experiments = [Ballpushing_utils.Experiment(Datapath / f) for f in Folders]

In [ ]:
# Build the dataset

magnet_data = Ballpushing_utils.Dataset(Experiments)

print(magnet_data)

In [ ]:
start = 30 * 60

magnet_data.generate_dataset("summary", time_range=[start])

In [ ]:
# Get the data columns

data_columns = magnet_data.data.columns

data_columns

# Do some plotting

Now we're gonna plot some metrics and compare them between Magnet y and magnet n.

In [ ]:
importlib.reload(HoloviewsTemplates)

In [ ]:
Plot = HoloviewsTemplates.jitter_boxplot(magnet_data.data, "TimeToFinish", "Magnet", magnet_data.metadata)

In [ ]:
Plot
# + Histogram distribution

## Make and save all metrics plots

In [ ]:
savepath = Utils.get_labserver() / "Experimental_data/MultiMazeRecorder/Plots/MagnetBlock/240613"

In [ ]:
# Generate the list of metrics

metrics = [
    "NumberEvents",
    "FinalEvent",
    "FinalTime",
    "SignificantEvents",
    "SignificantFirst",
    "SignificantFirstTime",
    "Pushes",
    "Pulls",
    "PullingRatio",
    "InteractionProportion",
    "AhaMoment",
    "AhaMomentIndex",
    "InsightEffect",
    "TimeToFinish",
    "SignificantRatio",
]

In [ ]:
# For each metric, generate a jitter boxplot and save it

for metric in metrics: 
    Plot = HoloviewsTemplates.jitter_boxplot(
        magnet_data.data, metric, "Magnet", magnet_data.metadata, scale_max=[True if metric == "FinalTime" else False]
    )
    hv.save(Plot, savepath / f"{metric}.html")

In [ ]:
# Make a nice layout with all the plots:
plots = []

for metric in metrics:
    Plot = HoloviewsTemplates.jitter_boxplot(
        magnet_data.data,
        metric,
        "Magnet",
        magnet_data.metadata,
        scale_max=[True if metric == "FinalTime" else False],
    )
    plots.append(Plot)

layout = hv.Layout(plots).cols(2)

In [ ]:
hv.save(layout, savepath / "All_metrics.html")

In [ ]:
importlib.reload(HoloviewsTemplates)

# Distribution

We can see that there seems to be some distribution variability between groups, which would be better highlighted by using something like an histogram

In [ ]:
Hist = HoloviewsTemplates.histograms(magnet_data.data, "TimeToFinish", "Magnet", bins = 20)

In [ ]:
Hist